In [1]:
import os

import pandas

from scripts.utility.azure_table_adaper import TableBroker

In [2]:
table_broker: TableBroker = TableBroker()

In [3]:
data_frame = pandas.DataFrame.from_dict(list(table_broker.get_all_entities("training")))

display(data_frame)
print(data_frame.shape)

,PartitionKey,RowKey,author,caption,flair,hash,id,image,permalink,subreddit,text,url
0,training,1000d16,princessxo699,A little girl in a pink dress is holding a pin...,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\reddit_images\GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,https://i.imgur.com/GgFEagO.jpg
1,training,1000fg0,BlkBrd1312,A woman in a pink bathing suit holds a baby in...,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\reddit_images\nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,HotGirlNextDoor,(IKTR),https://i.redd.it/nwa7hts2jb9a1.jpg
2,training,1000glf,toolate_sharkbait,A woman in a pink dress is smiling at the camera.,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\reddit_images\4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,AmIhotAF,Just looking for entertainment,https://i.redd.it/4xyb1vgbjb9a1.jpg
3,training,1000qiq,fireybubbulegum,A woman in a red dress is talking to a man in ...,NaN,7b0be38a08e3316051c6e5a3753bc791,1000qiq,D:\data\reddit_images\0mgrt95plb9a1.jpg,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,SFWRedheads,Who’s going to be my new years kiss tonight? 🤧,https://i.redd.it/0mgrt95plb9a1.jpg
4,training,1001fon,glam_pire,A woman in a red dress is talking to a man in ...,NaN,feed678c3b74ed1d0b7011f28e8f283f,1001fon,D:\data\reddit_images\gp4fhsdorb9a1.jpg,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,SFWRedheads,honk if you like my sweater 😁💕,https://i.redd.it/gp4fhsdorb9a1.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
4540,training,zzxyrq,Redredhed,A woman in a pink shirt and black pants is tal...,NaN,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,D:\data\reddit_images\eu6z40qyya9a1.jpg,/r/SFWRedheads/comments/zzxyrq/starting_the_da...,SFWRedheads,Starting the day with mushroom water in my mus...,https://i.redd.it/eu6z40qyya9a1.jpg
4541,training,zzy3sq,New-Cartographer3037,A girl in a pink dress is talking to another g...,Just Feelin' Cute 💕,d159090b1c7b485347cec61c70bc9c54,zzy3sq,D:\data\reddit_images\Hzad3q2.jpg,/r/SFWNextDoorGirls/comments/zzy3sq/your_new_y...,SFWNextDoorGirls,your new years resolution should be sleeping w...,https://i.imgur.com/Hzad3q2.jpg
4542,training,zzyerp,Juliadomaina,A girl in a red jacket is sitting on a red horse.,Female (18+),9279353b1761d62e7f8ca587a4146abf,zzyerp,D:\data\reddit_images\hgktc1on2b9a1.jpg,/r/AmIhotAF/comments/zzyerp/good_evening_every...,AmIhotAF,Good evening everyone ❤️,https://i.redd.it/hgktc1on2b9a1.jpg
4543,training,zzzg13,AdConsistent3184,A little girl in a pink dress is sitting on a ...,NaN,ba89401054b09d3ff2b608705f8a7110,zzzg13,D:\data\reddit_images\3rv0ql34bb9a1.jpg,/r/SFWRedheads/comments/zzzg13/cute_but_deadly/,SFWRedheads,Cute but Deadly 😋,https://i.redd.it/3rv0ql34bb9a1.jpg


In [4]:
from typing import Union
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

i = 0
def predict_step(image_path) -> Union[str, None]:
	try:
		global i
		i += 1
		total = 2567

		print(f":: {i}/{total} Completed\t{round(i/total*100, 2)}%")
		print(f":: Processioning {image_path}")

		device = torch.device("cpu")

		model.to(device)

		max_length = 32

		num_beams = 4

		gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

		images = []

		i_image = Image.open(image_path)
		if i_image.mode != "RGB":
			i_image = i_image.convert(mode="RGB")

		images.append(i_image)

		print(f":: Predicting image: {image_path}")

		pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values

		pixel_values = pixel_values.to(device)

		output_ids = model.generate(pixel_values, **gen_kwargs)

		print(f":: Decoding output for image: {image_path}")
		predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

		prediction = [prediction.strip() for prediction in predictions]

		print(f":: Completed prediction for image: {image_path}")
		if len(prediction) > 0:
			return prediction[0]
		else:
			return None

	except Exception as e:
		print(f":: Process Failed For {image_path} with {e}")
		return None

In [5]:
def filter_removed_images(row):
	if os.path.exists(row['image']):
		return True
	else:
		return False

In [6]:
def predict(row):
	caption = predict_step(row['image'])
	if caption is not None:
		return caption
	else:
		return None

In [7]:
filtered_data = data_frame.apply(filter_removed_images, axis=1)

filtered_data_frame = pandas.DataFrame(filtered_data, columns=['exits'])

combined_data = pandas.concat([data_frame, filtered_data_frame], axis=1)

df = combined_data.loc[combined_data.exits, :]

df.reindex()

,PartitionKey,RowKey,author,caption,flair,hash,id,image,permalink,subreddit,text,url,exits
0,training,1000d16,princessxo699,A little girl in a pink dress is holding a pin...,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\reddit_images\GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,https://i.imgur.com/GgFEagO.jpg,True
1,training,1000fg0,BlkBrd1312,A woman in a pink bathing suit holds a baby in...,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\reddit_images\nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,HotGirlNextDoor,(IKTR),https://i.redd.it/nwa7hts2jb9a1.jpg,True
2,training,1000glf,toolate_sharkbait,A woman in a pink dress is smiling at the camera.,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\reddit_images\4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,AmIhotAF,Just looking for entertainment,https://i.redd.it/4xyb1vgbjb9a1.jpg,True
5,training,1001ghp,eromorphic,A girl in a pink bathing suit walks down the b...,NaN,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,D:\data\reddit_images\v3ra9g4vrb9a1.jpg,/r/sfwpetite/comments/1001ghp/braids/,sfwpetite,Braids,https://i.redd.it/v3ra9g4vrb9a1.jpg,True
8,training,1001ssd,CertifiedDrinkSpill,A woman in a bikini and a man in a white t-shi...,NaN,a0b0ffef01a7abf9de351000026d49f0,1001ssd,D:\data\reddit_images\ldz8tgpsub9a1.jpg,/r/sfwpetite/comments/1001ssd/incredible_body/,sfwpetite,Incredible body,https://i.redd.it/ldz8tgpsub9a1.jpg,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,training,zzxq2l,PsychPsych19,A girl in a pink dress is standing in front of...,HotGirlNextDoor,d273d2f322103b2a431e9414e6443dfc,zzxq2l,D:\data\reddit_images\dozyybyzwa9a1.jpg,/r/HotGirlNextDoor/comments/zzxq2l/perfect_day...,HotGirlNextDoor,Perfect Day (IKTR),https://i.redd.it/dozyybyzwa9a1.jpg,True
4540,training,zzxyrq,Redredhed,A woman in a pink shirt and black pants is tal...,NaN,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,D:\data\reddit_images\eu6z40qyya9a1.jpg,/r/SFWRedheads/comments/zzxyrq/starting_the_da...,SFWRedheads,Starting the day with mushroom water in my mus...,https://i.redd.it/eu6z40qyya9a1.jpg,True
4541,training,zzy3sq,New-Cartographer3037,A girl in a pink dress is talking to another g...,Just Feelin' Cute 💕,d159090b1c7b485347cec61c70bc9c54,zzy3sq,D:\data\reddit_images\Hzad3q2.jpg,/r/SFWNextDoorGirls/comments/zzy3sq/your_new_y...,SFWNextDoorGirls,your new years resolution should be sleeping w...,https://i.imgur.com/Hzad3q2.jpg,True
4542,training,zzyerp,Juliadomaina,A girl in a red jacket is sitting on a red horse.,Female (18+),9279353b1761d62e7f8ca587a4146abf,zzyerp,D:\data\reddit_images\hgktc1on2b9a1.jpg,/r/AmIhotAF/comments/zzyerp/good_evening_every...,AmIhotAF,Good evening everyone ❤️,https://i.redd.it/hgktc1on2b9a1.jpg,True


In [8]:
import dask.dataframe as dd
ddf = dd.from_pandas(df, npartitions=4)

In [9]:
df['updated_caption'] = ddf.apply(predict, axis=1, meta=('updated_caption', 'str')).compute()

Predicting image 1 of 2567
list index out of range
Failed to update caption for image: D:\data\reddit_images\GgFEagO.jpg
Predicting image 2 of 2567
Predicting image 3 of 2567
list index out of range
Failed to update caption for image: D:\data\reddit_images\3fv37wf08yea1.jpg
Predicting image 4 of 2567
Predicting image 5 of 2567
list index out of range
Failed to update caption for image: D:\data\reddit_images\c6iacgmh8dy91.jpg
Predicting image 6 of 2567
Predicting image 7 of 2567
list index out of range
Failed to update caption for image: D:\data\reddit_images\EKaTETR.jpg
Predicting image 8 of 2567
Predicting image: D:\data\reddit_images\nwa7hts2jb9a1.jpg
Predicting image: D:\data\reddit_images\9EtCWNc.jpg
Predicting image: D:\data\reddit_images\igaTn9L.jpg
Predicting image: D:\data\reddit_images\7mL3bgp.jpg
Decoding output
Completed prediction for image: D:\data\reddit_images\igaTn9L.jpg
Updating caption for image: D:\data\reddit_images\igaTn9L.jpg to: a woman in a black shirt is lookin

D:\workspaces\General\venv\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (101461248 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Decoding output
Completed prediction for image: D:\data\reddit_images\nqkho3ba9yba1.jpg
Updating caption for image: D:\data\reddit_images\nqkho3ba9yba1.jpg to: a woman in a red shirt smiling and holding a phone
Predicting image 1211 of 2567
list index out of range
Failed to update caption for image: D:\data\reddit_images\rol0atz2hzba1.jpg
Predicting image 1212 of 2567
Predicting image: D:\data\reddit_images\gmm3w222kzba1.jpg
Decoding output
Completed prediction for image: D:\data\reddit_images\QCXfjoM.jpg
Updating caption for image: D:\data\reddit_images\QCXfjoM.jpg to: a beautiful young woman in a black dress posing for a picture
Predicting image 1213 of 2567
list index out of range
Failed to update caption for image: D:\data\reddit_images\6lx0j5jbh41a1.jpg
Predicting image 1214 of 2567
Predicting image: D:\data\reddit_images\ype28q5dh41a1.jpg
Decoding output
Completed prediction for image: D:\data\reddit_images\etue4ebd2n6a1.jpg
Updating caption for image: D:\data\reddit_images\etue4

C:\Users\AJ Stangl\AppData\Local\Temp\ipykernel_12556\2863407505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['updated_caption'] = ddf.apply(predict, axis=1, meta=('updated_caption', 'str')).compute()


In [10]:
display(df)

,PartitionKey,RowKey,author,caption,flair,hash,id,image,permalink,subreddit,text,url,exits,updated_caption
0,training,1000d16,princessxo699,A little girl in a pink dress is holding a pin...,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\reddit_images\GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,https://i.imgur.com/GgFEagO.jpg,True,None
1,training,1000fg0,BlkBrd1312,A woman in a pink bathing suit holds a baby in...,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\reddit_images\nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,HotGirlNextDoor,(IKTR),https://i.redd.it/nwa7hts2jb9a1.jpg,True,a woman in a bathing suit with a cell phone in...
2,training,1000glf,toolate_sharkbait,A woman in a pink dress is smiling at the camera.,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\reddit_images\4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,AmIhotAF,Just looking for entertainment,https://i.redd.it/4xyb1vgbjb9a1.jpg,True,None
5,training,1001ghp,eromorphic,A girl in a pink bathing suit walks down the b...,NaN,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,D:\data\reddit_images\v3ra9g4vrb9a1.jpg,/r/sfwpetite/comments/1001ghp/braids/,sfwpetite,Braids,https://i.redd.it/v3ra9g4vrb9a1.jpg,True,a beautiful young woman in a bikini posing for...
8,training,1001ssd,CertifiedDrinkSpill,A woman in a bikini and a man in a white t-shi...,NaN,a0b0ffef01a7abf9de351000026d49f0,1001ssd,D:\data\reddit_images\ldz8tgpsub9a1.jpg,/r/sfwpetite/comments/1001ssd/incredible_body/,sfwpetite,Incredible body,https://i.redd.it/ldz8tgpsub9a1.jpg,True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,training,zzxq2l,PsychPsych19,A girl in a pink dress is standing in front of...,HotGirlNextDoor,d273d2f322103b2a431e9414e6443dfc,zzxq2l,D:\data\reddit_images\dozyybyzwa9a1.jpg,/r/HotGirlNextDoor/comments/zzxq2l/perfect_day...,HotGirlNextDoor,Perfect Day (IKTR),https://i.redd.it/dozyybyzwa9a1.jpg,True,a woman posing for a picture while holding a s...
4540,training,zzxyrq,Redredhed,A woman in a pink shirt and black pants is tal...,NaN,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,D:\data\reddit_images\eu6z40qyya9a1.jpg,/r/SFWRedheads/comments/zzxyrq/starting_the_da...,SFWRedheads,Starting the day with mushroom water in my mus...,https://i.redd.it/eu6z40qyya9a1.jpg,True,None
4541,training,zzy3sq,New-Cartographer3037,A girl in a pink dress is talking to another g...,Just Feelin' Cute 💕,d159090b1c7b485347cec61c70bc9c54,zzy3sq,D:\data\reddit_images\Hzad3q2.jpg,/r/SFWNextDoorGirls/comments/zzy3sq/your_new_y...,SFWNextDoorGirls,your new years resolution should be sleeping w...,https://i.imgur.com/Hzad3q2.jpg,True,a young woman in a red dress is talking on the...
4542,training,zzyerp,Juliadomaina,A girl in a red jacket is sitting on a red horse.,Female (18+),9279353b1761d62e7f8ca587a4146abf,zzyerp,D:\data\reddit_images\hgktc1on2b9a1.jpg,/r/AmIhotAF/comments/zzyerp/good_evening_every...,AmIhotAF,Good evening everyone ❤️,https://i.redd.it/hgktc1on2b9a1.jpg,True,None


In [15]:
records = df.to_dict('records')

total_records = len(records)

for i, record in enumerate(records):
	if i % 10 == 0:
		print(f":: {i}/{total_records} Completed\t{round(i/total_records*100, 2)}%")
	table_broker.upsert_entity_to_table("training", record)

In [16]:
data_frame = pandas.DataFrame.from_dict(list(table_broker.get_all_entities("training")))

display(data_frame)

,PartitionKey,RowKey,author,caption,exits,flair,hash,id,image,permalink,subreddit,text,url,updated_caption
0,training,1000d16,princessxo699,A little girl in a pink dress is holding a pin...,True,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\reddit_images\GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,https://i.imgur.com/GgFEagO.jpg,NaN
1,training,1000fg0,BlkBrd1312,A woman in a pink bathing suit holds a baby in...,True,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\reddit_images\nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,HotGirlNextDoor,(IKTR),https://i.redd.it/nwa7hts2jb9a1.jpg,a woman in a bathing suit with a cell phone in...
2,training,1000glf,toolate_sharkbait,A woman in a pink dress is smiling at the camera.,True,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\reddit_images\4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,AmIhotAF,Just looking for entertainment,https://i.redd.it/4xyb1vgbjb9a1.jpg,NaN
3,training,1000qiq,fireybubbulegum,A woman in a red dress is talking to a man in ...,NaN,NaN,7b0be38a08e3316051c6e5a3753bc791,1000qiq,D:\data\reddit_images\0mgrt95plb9a1.jpg,/r/SFWRedheads/comments/1000qiq/whos_going_to_...,SFWRedheads,Who’s going to be my new years kiss tonight? 🤧,https://i.redd.it/0mgrt95plb9a1.jpg,NaN
4,training,1001fon,glam_pire,A woman in a red dress is talking to a man in ...,NaN,NaN,feed678c3b74ed1d0b7011f28e8f283f,1001fon,D:\data\reddit_images\gp4fhsdorb9a1.jpg,/r/SFWRedheads/comments/1001fon/honk_if_you_li...,SFWRedheads,honk if you like my sweater 😁💕,https://i.redd.it/gp4fhsdorb9a1.jpg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4540,training,zzxyrq,Redredhed,A woman in a pink shirt and black pants is tal...,True,NaN,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,D:\data\reddit_images\eu6z40qyya9a1.jpg,/r/SFWRedheads/comments/zzxyrq/starting_the_da...,SFWRedheads,Starting the day with mushroom water in my mus...,https://i.redd.it/eu6z40qyya9a1.jpg,NaN
4541,training,zzy3sq,New-Cartographer3037,A girl in a pink dress is talking to another g...,True,Just Feelin' Cute 💕,d159090b1c7b485347cec61c70bc9c54,zzy3sq,D:\data\reddit_images\Hzad3q2.jpg,/r/SFWNextDoorGirls/comments/zzy3sq/your_new_y...,SFWNextDoorGirls,your new years resolution should be sleeping w...,https://i.imgur.com/Hzad3q2.jpg,a young woman in a red dress is talking on the...
4542,training,zzyerp,Juliadomaina,A girl in a red jacket is sitting on a red horse.,True,Female (18+),9279353b1761d62e7f8ca587a4146abf,zzyerp,D:\data\reddit_images\hgktc1on2b9a1.jpg,/r/AmIhotAF/comments/zzyerp/good_evening_every...,AmIhotAF,Good evening everyone ❤️,https://i.redd.it/hgktc1on2b9a1.jpg,NaN
4543,training,zzzg13,AdConsistent3184,A little girl in a pink dress is sitting on a ...,NaN,NaN,ba89401054b09d3ff2b608705f8a7110,zzzg13,D:\data\reddit_images\3rv0ql34bb9a1.jpg,/r/SFWRedheads/comments/zzzg13/cute_but_deadly/,SFWRedheads,Cute but Deadly 😋,https://i.redd.it/3rv0ql34bb9a1.jpg,NaN


(4545, 14)
